In [2]:
from __future__ import division
import tensorflow as tf
import logging
import argparse
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD, Adam
import string
import os
import pandas as pd
import random
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.models import load_model
import shutil
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, add, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import yaml
import tensorflow.keras as keras
import sys
from load_tf_record import TFRecordLoader
#sys.path.append('../')
from utils.dataset import get_dataset
########### Logger setup ##############
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

######### Load config ######################################
stream = open('config.yaml', 'r')
config_arg = yaml.safe_load(stream)
######## set argument ######################################
epochs = int(config_arg['training']['epoch'])
print('epochs == {}'.format(epochs))
batch_size = int(config_arg['training']['train_batch'])
val_batch_size = int(config_arg['training']['val_batch'])
model_version = str(config_arg['training']['version'])
loss = config_arg['training']['loss'] # binary_crossentropy or categorical_crossentropy
class_mode = config_arg['data_generator']['class_mode'] # binary or categorical
img_target_size = config_arg['image']['size']


MODEL_ARCHITECTURE = config_arg['type'] + model_version + ".json"
CHECK_POINT = config_arg['type'] + model_version + '_weights.{epoch:02d}-{loss:.2f}.hdf5'
MODEL_FILE = config_arg['type'] + model_version + '_model.h5'
WEIGHT_FILE = config_arg['type'] + model_version + '_weights_.h5'
HISTORY_FILE = 'history_' + config_arg['type'] + model_version + '.csv'
LR_FILE='lr_' + config_arg['type'] + model_version + '.csv'
platform = str(config_arg['platform'])
print("platform", platform=="gcp")
if platform == 'gcp':
    TRAIN_DIR = str(config_arg['data']['gcp']['train'])
    EVAL_DIR = str(config_arg['data']['gcp']['test'])
    destination = config_arg['save_model']['local']['path_prefix']
else:
    TRAIN_DIR = config_arg['data']['local']['train']
    EVAL_DIR = config_arg['data']['local']['test']
    destination = config_arg['save_model']['local']['path_prefix']




########### Training data generator ########
#FILENAMES_TRAIN = tf.io.gfile.glob("/home/omen/lab/GCP/Transfer_Learning/tfrecord/train*")
#FILENAMES_EVAL = tf.io.gfile.glob("/home/omen/lab/GCP/Transfer_Learning/tfrecord/test*")

#data_loader = TFRecordLoader(batch_size, img_target_size)
print(TRAIN_DIR)
print(EVAL_DIR)
train_generator = get_dataset(TRAIN_DIR, 'train', batch_size=batch_size)
eval_generator =  get_dataset(EVAL_DIR, 'test', batch_size=1)
debug=False
if debug:
    image_batch, label_batch = next(iter(train_generator))
    data_loader.show_batch(image_batch.numpy(), label_batch.numpy())
    print("train_generator: ", train_generator)
############ Define Model ##############
nb_epochs = epochs


# Create base model
basemodel = keras.applications.Xception(
    weights='imagenet',
    input_shape=(img_target_size, img_target_size, 3),
    include_top=False)

# Freeze base model
basemodel.trainable = False

# Create new model on top.
inputs = keras.Input(shape=(img_target_size, img_target_size, 3))
x = basemodel(inputs, training=False)

# Full connection
flatten = Flatten()(x) 
fc1 = Dense(units = 128, activation = 'relu')(flatten)
fc2_out = Dense(units = 1, activation = 'sigmoid')(fc1)

model = tf.keras.Model(inputs, fc2_out)

#for layer in basemodel.layers:
    #layer.trainable = False


opt = Adam(lr=5e-4, decay=0.1)
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss="binary_crossentropy",
        metrics=['accuracy'],
    )

logger.debug("Model summary...")
model.count_params()
model.summary()



############# Support infra #########
checkpoint_path = None
log_path = None

dirpath = os.getcwd()
print("current directory is : " + dirpath)
destination = destination + model_version
if os.path.isdir(destination):
    shutil.rmtree(destination, ignore_errors = True)
    logger.info("Removed old {}".format(destination))
os.makedirs(destination)
checkpoint_path = destination + "/checkpoints"
os.mkdir(checkpoint_path)
log_path = destination + "/logs"
os.mkdir(log_path)
logger.info("Finished creating {}".format(destination))


# Write model architecture
with open(os.path.join(destination , MODEL_ARCHITECTURE), "w") as f:
        f.write(model.to_json())

run_id = "cat_dog-" + str(batch_size) + "-" + '' \
.join(random
      .SystemRandom()
      .choice(string.ascii_uppercase) for _ in range(10)
)


from keras.callbacks import LearningRateScheduler

def schedule(epoch, lr):
    if epoch%25 == 0:
        return lr * 0.5
    return lr
#lr_scheduler = LearningRateScheduler(schedule, verbose=0)

callbacks = [
    ModelCheckpoint(
        os.path.join(checkpoint_path, CHECK_POINT),
        monitor="val_loss",
        verbose=1,
        save_best_only=True,
        mode="auto"
        ),
    TensorBoard(log_dir= os.path.join(log_path,run_id) ),
    #lr_scheduler,
    ]

########### Run the model #############
hist = model.fit(
train_generator,
callbacks = callbacks,
batch_size=16,
steps_per_epoch = 10,# there are around 9776 images, % by batch size of 16
epochs=nb_epochs,
validation_data=eval_generator,
shuffle=True,
validation_steps=2,
#verbose=2,
)

## Print lr ###
#print(lr_scheduler.history)

######## Save Model ###############
logger.info("save model")
model.save(os.path.join(destination, MODEL_FILE))
model.save_weights(os.path.join(destination, WEIGHT_FILE), overwrite=True)
pd.DataFrame(hist.history).to_csv(os.path.join(destination,HISTORY_FILE))


epochs == 2
platform True
gs://kubeflow-test-288607-kubeflowpipelines-default/transfer-learning-keras-training/data/train
gs://kubeflow-test-288607-kubeflowpipelines-default/transfer-learning-keras-training/data/test
tf.Tensor(
[b'gs://kubeflow-test-288607-kubeflowpipelines-default/transfer-learning-keras-training/data/train/train-00000-of-00002'
 b'gs://kubeflow-test-288607-kubeflowpipelines-default/transfer-learning-keras-training/data/train/train-00001-of-00002'], shape=(2,), dtype=string)
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.


Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.


Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.


Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.


tf.Tensor(
[b'gs://kubeflow-test-288607-kubeflowpipelines-default/transfer-learning-keras-training/data/test/test-00000-of-00002'
 b'gs://kubeflow-test-288607-kubeflowpipelines-default/transfer-learning-keras-training/data/test/test-00001-of-00002'], shape=(2,), dtype=string)


INFO:__main__:Finished creating ./models/2


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               12845184  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 33,706,793
Trainable params: 12,845,313
Non-trainable params: 20,861,480
_________________________________________________________________
current directory is : /home/omen/lab/GCP/Transfer_Learning/transfer_learning_with_keras
Epoch 1/2


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


10/10 [==============================] - ETA: 0s - loss: 31.5271 - accuracy: 0.8500
Epoch 00001: val_loss improved from inf to 0.00000, saving model to ./models/2/checkpoints/cat_dog_clf2_weights.01-31.53.hdf5
10/10 [==============================] - 6s 570ms/step - loss: 31.5271 - accuracy: 0.8500 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/2
10/10 [==============================] - ETA: 0s - loss: 8.3726 - accuracy: 0.9375
Epoch 00002: val_loss did not improve from 0.00000
10/10 [==============================] - 5s 500ms/step - loss: 8.3726 - accuracy: 0.9375 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


INFO:__main__:save model


In [8]:
%load_ext tensorboard
%tensorboard --logdir ./models/2/logs/cat_dog-8-FFXEERWOXY/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
